In [1]:
from src.ApylabradosModule import Pawns,Word,FrequencyTable,Board,Dictionary
import src.ApylabradosModule as ApylabradosModule
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ApylabradosModule.startGame()
while not ApylabradosModule.end and ApylabradosModule.bag_of_pawns.getTotalPawns()>0:
    ApylabradosModule.showOption()
if ApylabradosModule.bag_of_pawns.getTotalPawns()<=0:
    print("¡¡¡Te has quedado sin fichas!!!")
    print("Fin del juego")
print(f"has conseguido {Board.score} de puntos en esta partida")

¿Que deseas hacer ?  
 
 Para continuar, introduce: ENTERWORD
 Para consultar fichar, introduce: MYPAWNS
 Para consultar puntos actuales, introduce: MYSCORE
 Para consultar puntos de cada ficha, introduce: PAWNSPOINTS
 Para obtener Ayuda con la posible palabra, introduce: HELPWORD
 Para obtener Ayuda con la Legenda, introduce: HELPLEGEND
 Para Salir del juego, introduce: QUITGAME
 
Fin del Juego
has conseguido 0 de puntos en esta partida


In [10]:
import gradio as gr
import src.ApylabradosModule as ApylabradosModule

# Inicialización del juego
ApylabradosModule.startGame()

def jugar_accion(accion, palabra=None, fila=None, columna=None, direccion=None):
    if accion == "ENTERWORD":
        try:
            # Validar que todos los campos estén completos
            if not palabra or fila is None or columna is None or not direccion:
                return "Por favor, completa todos los campos", "images/tablero.png"
            
            # Convertir la palabra a mayúsculas y crear objeto Word
            palabra_upper = palabra.upper().strip()
            word_obj = ApylabradosModule.Word()
            for letra in palabra_upper:
                word_obj.word.append(letra)
            
            # Validar que la palabra existe en el diccionario
            if not ApylabradosModule.Dictionary.validateWord(word_obj):
                return "La palabra no existe en el diccionario", "images/tablero.png"
            
            # Validar que se puede formar con las fichas disponibles
            word_freq = word_obj.getFrequency()
            player_freq = ApylabradosModule.player_pawns.getFrequency()
            
            if not ApylabradosModule.FrequencyTable.isSubset(word_freq, player_freq):
                return "No puedes formar esta palabra con tus fichas", "images/tablero.png"
            
            # Validar que se puede colocar en la posición especificada
            posible, mensaje = ApylabradosModule.board.isPossible(word_obj, int(fila), int(columna), direccion)
            if not posible:
                return f"Error: {mensaje}", "images/tablero.png"
            
            # Colocar la palabra en el tablero
            ApylabradosModule.board.placeWord(ApylabradosModule.player_pawns, word_obj, int(fila), int(columna), direccion)
            
            # Generar nueva imagen del tablero
            nueva_imagen = ApylabradosModule.board.showBoard()
            
            return f"¡Palabra '{palabra_upper}' colocada exitosamente! Puntuación: {ApylabradosModule.Board.score}", nueva_imagen
            
        except Exception as e:
            return f"Error: {str(e)}", "images/tablero.png"
    
    elif accion == "HELPLEGEND":
        return "Leyenda del tablero mostrada", "images/legend.png"
    elif accion == "QUITGAME":
        return "¡Gracias por jugar!", "images/tablero.png"
    else:
        return "Opción no reconocida", "images/tablero.png"
def palabras_sugeridas():
    # Lógica para obtener palabras sugeridas
    # Usamos la función showWords pero adaptada para devolver string
    from io import StringIO
    import sys
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    ApylabradosModule.Dictionary.showWords(ApylabradosModule.player_pawns)
    sys.stdout = old_stdout
    resultado = mystdout.getvalue()
    if not resultado.strip():
        resultado = "No hay palabras sugeridas."
    return resultado
with gr.Blocks() as demo:
    gr.Markdown("## Apailabrados")
    with gr.Row():
        # Columna izquierda: interacción con Tabs
        with gr.Column(scale=1):
            with gr.Tabs():
                with gr.TabItem("Instrucciones"):
                    gr.Markdown(ApylabradosModule.welcome())
                    gr.Markdown(ApylabradosModule.instrucciones())
                with gr.TabItem("Mis fichas"):
                    pawns_box = gr.Markdown(value=ApylabradosModule.player_pawns.getPawnsString())
                    helpword_btn = gr.Button("HELPWORD")
                    sugeridas_box = gr.Textbox(label="Palabras sugeridas", lines=5, interactive=False)
                    helpword_btn.click(palabras_sugeridas, outputs=sugeridas_box)
                    # Aquí podrías actualizar el contenido con la función correspondiente
                with gr.TabItem("Jugar"):
                    accion = gr.Dropdown(
                        choices=[
                            "ENTERWORD",
                             "HELPLEGEND", "QUITGAME"
                        ],
                        value='ENTERWORD',
                        label="¿Qué deseas hacer?"
                    )
                    palabra = gr.Textbox(label="Palabra (si aplica)", visible=False)
                    fila = gr.Number(label="Fila (si aplica)", visible=False)
                    columna = gr.Number(label="Columna (si aplica)", visible=False)
                    direccion = gr.Dropdown(choices=["V", "H"], label="Dirección (si aplica)", visible=False)
                    btn = gr.Button("Enviar")
                    salida = gr.Textbox(label="Salida")
        # Columna derecha: tablero
        with gr.Column(scale=2):
            tablero = gr.Image(value="images/tablero.png", label="Tablero")
            legenda = gr.Image(value='images/legend.png', label='legend')

    # Mostrar u ocultar campos según la acción seleccionada
    def mostrar_campos(accion):
        if accion == "ENTERWORD":
            return {palabra: gr.update(visible=True),
                    fila: gr.update(visible=True),
                    columna: gr.update(visible=True),
                    direccion: gr.update(visible=True)}
        else:
            return {palabra: gr.update(visible=False),
                    fila: gr.update(visible=False),
                    columna: gr.update(visible=False),
                    direccion: gr.update(visible=False)}
    # Ejecutar mostrar_campos al inicio para reflejar el estado inicial
    demo.load(lambda: mostrar_campos("ENTERWORD"), None, [palabra, fila, columna, direccion])
    accion.change(mostrar_campos, inputs=accion, outputs=[palabra, fila, columna, direccion])
    btn.click(jugar_accion, inputs=[accion, palabra, fila, columna, direccion], outputs=[salida, tablero])

demo.launch()


* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
